# HydroSOS - Persistence forecast method

In this notebook, we explore the Persistance and Hydrological Analogue method

## Import the python libraries

In [1]:
%reset -f

In [2]:
# Importing the libraries
import os
import pandas as pd
import seaborn as sns
import numpy as np
import calendar
import datetime
import matplotlib.pyplot as plt
from scipy import stats
plt.style.use('classic')
%matplotlib inline

from IPython.display import HTML

sns.set()

In [3]:
# install a conda package in the current Jupyter kernel
#import sys
#!conda install --yes --prefix {sys.prefix} missingno

In [4]:
import missingno

### We create function for percentile definition

In [5]:
# Quantile Function Definitions
def q1(x):
    return x.quantile(0.28)

def q2(x):
    return x.median()

def q3(x):
    return x.quantile(0.72)

def q5(x):
    return x.quantile(0.05)

def q95(x):
    return x.quantile(0.95)

def q87(x):
    return x.quantile(0.87)

def q13(x):
    return x.quantile(0.13)


## Import Data 

In [17]:
MONTHLY_ANOMALY = pd.read_csv('../data/santalucia_caudales_abr2023_MA.csv',parse_dates=['Month'],index_col="Month",na_values="")

In [18]:
DISCHARGE_MONTHLY = pd.read_csv('../data/santalucia_caudales_abr2023_DA.csv',parse_dates=['Fecha'],index_col="Fecha",na_values="")

In [19]:
YEAR_HINDCAST_SELECTED = np.unique(DISCHARGE_MONTHLY['Year'])
# rename columns change str for int
MONTHLY_ANOMALY.columns = YEAR_HINDCAST_SELECTED

In [20]:
range_analysis = range(YEAR_HINDCAST_SELECTED[0],YEAR_HINDCAST_SELECTED[-1])

In [21]:
LOG_DISCHARGE_MONTHLY_PIVOT = pd.pivot_table(DISCHARGE_MONTHLY, index=['Month'],columns=['Year'], values=['Q_to_log'],dropna=False)
LOG_DISCHARGE_MONTHLY_PIVOT.columns = YEAR_HINDCAST_SELECTED

In [22]:
ANOMALY_PERSISTENCE = MONTHLY_ANOMALY[range_analysis]
QLOG_PERSISTENCE = LOG_DISCHARGE_MONTHLY_PIVOT[range_analysis]
HTML(ANOMALY_PERSISTENCE.to_html())

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,-1.217702,-0.332189,-0.151861,-0.391850,1.297849,-0.771544,-0.773628,-0.609809,1.599805,2.014703,-0.752051,NaN,0.168765,1.662446,0.220363,0.214728,2.427332,-0.405921,-0.069919,-0.275124,1.082403,-0.171486,NaN,0.908379,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,0.695973,-0.834223,-0.451418,1.702978,NaN,0.080346,-1.114598,-0.735161,-0.535340,2.378751,-1.082684,NaN,-0.004572,0.958376,-0.386063,0.492906,-0.137558,-0.672549,-0.303736,0.213084,2.828374,-0.399055,NaN,-0.278387,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,0.368636,-0.618578,-0.621814,-0.434609,0.184724,-0.028342,-1.316506,-0.671217,-0.821700,1.549766,-1.341889,1.498400,2.527140,0.214817,-0.607739,-0.278698,0.094414,2.332151,0.803340,0.482773,0.549333,-0.536844,NaN,NaN,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,1.740413,-0.820711,-0.109710,0.941890,-0.084077,1.142900,0.569727,-1.152541,-0.223646,0.620392,NaN,-0.303012,2.078428,-0.689754,-0.127231,1.688282,-0.487829,1.956936,-0.653459,-0.651857,0.446257,-0.679586,-0.586552,NaN,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,0.055118,-0.026225,0.897623,0.929453,1.142730,0.193611,-0.909332,-1.229191,-0.437847,-0.628618,2.064755,-0.136805,1.454932,0.670987,0.324343,1.216989,-0.986345,1.713232,-0.864665,-1.028210,-0.305526,-0.990349,-1.220725,NaN,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,0.182126,0.362392,0.517625,0.271876,-0.628118,0.309844,-1.732159,-0.216433,-0.312289,0.386102,0.505539,1.243150,0.734981,1.084724,-0.109196,1.533202,1.317109,-0.217848,-0.893280,-0.582289,0.612672,NaN,-1.206965,-1.173551,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,-1.619038,0.623227,0.285137,-0.455937,-0.508600,0.499323,-2.199121,-1.788752,0.929367,1.672127,1.241263,-0.321207,0.434812,0.506555,-0.236939,0.311917,0.214526,-1.384830,-0.367381,1.067286,1.057527,NaN,-0.782419,NaN,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,-1.997518,0.865586,-0.708144,0.028734,0.605638,-1.407229,-2.770442,-0.163174,-0.543317,-0.011291,NaN,0.712015,0.528228,0.476646,-0.759065,-1.105045,0.331471,-0.278306,-1.034313,-0.365984,0.993314,NaN,NaN,NaN,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,-1.517861,0.693688,0.532099,0.656601,0.844961,-1.777712,0.490324,-1.238533,-0.830146,0.421248,NaN,0.039467,0.323237,1.584296,0.270234,-0.408260,-1.201608,1.047919,-0.988840,-0.336547,1.746241,NaN,NaN,NaN,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643


In [23]:
ANOMALY_PERSISTENCE_TS = pd.melt(ANOMALY_PERSISTENCE.reset_index(), id_vars='Month',var_name=['Year'],ignore_index = True)
ANOMALY_PERSISTENCE_TS.columns = ['Month','Year','Anomaly_Qlog']
HTML(ANOMALY_PERSISTENCE_TS.head(6).to_html())

,Month,Year,Anomaly_Qlog
0,1,1980,-2.123750
1,2,1980,0.316909
2,3,1980,1.120851
3,4,1980,1.200881
4,5,1980,1.373637
5,6,1980,0.796236


In [24]:
QLOG_PERSISTENCE_TS = pd.melt(QLOG_PERSISTENCE.reset_index(), id_vars='Month',var_name=['Year'],ignore_index = True)
QLOG_PERSISTENCE_TS.columns = ['Month','Year','Qlog']
HTML(QLOG_PERSISTENCE_TS.head(6).to_html())

,Month,Year,Qlog
0,1,1980,0.181165
1,2,1980,3.345911
2,3,1980,4.463403
3,4,1980,5.148583
4,5,1980,5.618806
5,6,1980,5.431652


In [25]:
ANOMALY_PERSISTENCE_TS['DATE'] = pd.to_datetime(ANOMALY_PERSISTENCE_TS[['Year', 'Month']].assign(DAY=1))
HTML(ANOMALY_PERSISTENCE_TS.head(6).to_html(index=False))

Month,Year,Anomaly_Qlog,DATE
1,1980,-2.123750,1980-01-01
2,1980,0.316909,1980-02-01
3,1980,1.120851,1980-03-01
4,1980,1.200881,1980-04-01
5,1980,1.373637,1980-05-01
6,1980,0.796236,1980-06-01


## Persistence forecast

We create a function called add_month that add the number of months based on an input date. This will be used in the Jack-Knife Validation

In [26]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

We import the monthly river flow anomalies from 1980 to 2023 

### Jack-knife validation

In this section, we will apply Jack Knife verification for each year and for each month of that year. The Jack Knife verification is a method that drop one year and calculate the forecast using the remaining of the years. 

Performance is based on “jack-knife” hindcasts for the period of observations available at each station. This involves dropping 1 year at a time from the record, and forecasting the flow anomalies in this missing year using the remaining years in the record. In this way, a hindcast will be made for each year which can be compared with the flow anomaly that was actually observed in that year. For each method, the correlation between its hindcast record and the observed record is calculated for each calendar month, and is used as a measure of its performance.

In [28]:
# Select the year of analysis
YEAR_ANALYSIS = range_analysis
MONTH_ANALYSIS = range(1,13,1)

# Create empty dataframe
ANOMALY_HINDCAST_PERSISTENCE = pd.DataFrame()
HINDCAST_MONTHS_PERSISTENCE = pd.DataFrame()
HINDCAST_STATS_QLOG_PERSISTENCE = pd.DataFrame()
HINDCAST_STATS_ANOMALY_PERSISTENCE = pd.DataFrame()
RESCALE_FORECAST_PERSISTENCE =  ANOMALY_PERSISTENCE_TS
SCALE_FORECAST_PERSISTENCE = ANOMALY_PERSISTENCE_TS

# Make the resample 
for y in YEAR_ANALYSIS:
    # extract the monthly anomalies for the corresponding year of analysis
    ANOMALY_YEAR = ANOMALY_PERSISTENCE[y].to_frame()

    # From the Qlog values, drop the monthly values for the year of analysis and mantain the remaining years. 
    # This is the Qlog hindcast series
    QLOG_HINDCAST_PERSISTENCE= QLOG_PERSISTENCE.drop([y],axis=1)
    QLOG_HINDCAST_PERSISTENCE_TS = QLOG_PERSISTENCE_TS[QLOG_PERSISTENCE_TS['Year'] != y]
    # From the monthly anomaly values, drop the monthly values for the year of analysis and mantain the remaining years. 
    # This is the anomaly hindcast series
    ANOMALY_HINDCAST_PERSISTENCE = ANOMALY_PERSISTENCE.drop([y],axis=1)
    # Calculate statistic for the hindcast Qlog serie
    HINDCAST_STATS_QLOG_PERSISTENCE['mean'] = np.mean(QLOG_HINDCAST_PERSISTENCE,axis=1)
    HINDCAST_STATS_QLOG_PERSISTENCE['std'] = np.std(QLOG_HINDCAST_PERSISTENCE,axis=1)
    # Calculate statistic for the hindcast Anomaly serie
    HINDCAST_STATS_ANOMALY_PERSISTENCE['mean'] = np.mean(ANOMALY_HINDCAST_PERSISTENCE,axis=1)
    HINDCAST_STATS_ANOMALY_PERSISTENCE['std'] = np.std(ANOMALY_HINDCAST_PERSISTENCE,axis=1)

    # for each year of analysis, we are going to iterate by each month (1 to 12)
    for m in MONTH_ANALYSIS:
        print('Dropping year:',y,"-","End-Month:",m)
        # extract from the hindcast series, all the corresponding monthly values based on the month of analysis. After this, it transpose the table  
        HINDCAST_MONTHS_PERSISTENCE = ANOMALY_HINDCAST_PERSISTENCE.loc[[m]].T
        # Obtain the Anomaly value for the end month (analysis)
        ANOMALY_ANALYSIS_PERSISTENCE = ANOMALY_YEAR.query('index==@m')[y].item()
        # Create a datetime that consider the current year and month
        END_MONTH = datetime.date.today().replace(year=y, month=m, day=1)
        # Using the add_month function, we add n lead month for the forecast
        ONE_MONTH = add_months(END_MONTH,1)
        TWO_MONTH = add_months(END_MONTH,2)
        THREE_MONTH = add_months(END_MONTH,3)
        SIX_MONTH = add_months(END_MONTH,6)
        # Do the forecast using persistence method. 
        SCALE_FORECAST_PERSISTENCE.loc[SCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'1_monthFcst'] = float(ANOMALY_ANALYSIS_PERSISTENCE)
        SCALE_FORECAST_PERSISTENCE.loc[SCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'2_monthFcst'] = float(ANOMALY_ANALYSIS_PERSISTENCE)
        SCALE_FORECAST_PERSISTENCE.loc[SCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'3_monthFcst'] = float(ANOMALY_ANALYSIS_PERSISTENCE)
        SCALE_FORECAST_PERSISTENCE.loc[SCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'6_monthFcst'] = float(ANOMALY_ANALYSIS_PERSISTENCE)
        # re-standardised all the month of analysis from the hindcast series based on the mean and standard deviation.
        ANOMALY_RESCALE_HINDCAST_PERSISTENCE = (HINDCAST_MONTHS_PERSISTENCE - np.mean(HINDCAST_MONTHS_PERSISTENCE,axis=0))/np.std(HINDCAST_MONTHS_PERSISTENCE,axis=0)
        # Re-standardised the qlog discharge anomaly of the current month 
        ANOMALY_RESCALE_ANALYSIS_PERSISTENCE = (ANOMALY_ANALYSIS_PERSISTENCE - np.mean(HINDCAST_MONTHS_PERSISTENCE,axis=0))/np.std(HINDCAST_MONTHS_PERSISTENCE,axis=0)
        # Store the rescale forecast, mean and std (1-month)
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'1_monthFcst'] = float(ANOMALY_RESCALE_ANALYSIS_PERSISTENCE)
        QLOG_MONTH_PERSISTENCE = QLOG_HINDCAST_PERSISTENCE_TS.query("Month == @ONE_MONTH.month")

        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'Log_Q_PRED_1M'] = float((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][ONE_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][ONE_MONTH.month])
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'Q_PRED_1M'] = float(np.exp((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][ONE_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][ONE_MONTH.month]))

        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'mean_A_H'] = float(np.mean(HINDCAST_MONTHS_PERSISTENCE,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'std_A_H'] = float(np.std(HINDCAST_MONTHS_PERSISTENCE,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'mean'] = float(np.mean(QLOG_MONTH_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'std'] = float(np.std(QLOG_MONTH_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'low'] = float(np.nanpercentile(QLOG_MONTH_PERSISTENCE['Qlog'],28,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'medium'] = float(np.nanpercentile(QLOG_MONTH_PERSISTENCE['Qlog'],50,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@ONE_MONTH.month & Year==@ONE_MONTH.year'),'high'] = float(np.nanpercentile(QLOG_MONTH_PERSISTENCE['Qlog'],72,axis=0))
        
        # Store the rescale forecast, mean and std (2-month)
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'2_monthFcst'] = float(ANOMALY_RESCALE_ANALYSIS_PERSISTENCE)
        QLOG_MONTH_02_PERSISTENCE = QLOG_HINDCAST_PERSISTENCE_TS.query("Month == @TWO_MONTH.month")
        
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'Log_Q_PRED_2M'] = float((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][ONE_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][TWO_MONTH.month])
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'Q_PRED_2M'] = float(np.exp((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][ONE_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][TWO_MONTH.month]))
        
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'mean'] = float(np.mean(QLOG_MONTH_02_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'std'] = float(np.std(QLOG_MONTH_02_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'low'] = float(np.nanpercentile(QLOG_MONTH_02_PERSISTENCE['Qlog'],28,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'medium'] = float(np.nanpercentile(QLOG_MONTH_02_PERSISTENCE['Qlog'],50,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@TWO_MONTH.month & Year==@TWO_MONTH.year'),'high'] = float(np.nanpercentile(QLOG_MONTH_02_PERSISTENCE['Qlog'],72,axis=0))
        
        # Store the rescale forecast, mean and std (3-month)
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'3_monthFcst'] = float(ANOMALY_RESCALE_ANALYSIS_PERSISTENCE)
        QLOG_MONTH_03_PERSISTENCE = QLOG_HINDCAST_PERSISTENCE_TS.query("Month == @THREE_MONTH.month")
        
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'Log_Q_PRED_3M'] = float((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][THREE_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][THREE_MONTH.month])
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'Q_PRED_3M'] = float(np.exp((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][THREE_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][THREE_MONTH.month]))
        
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'mean'] = float(np.mean(QLOG_MONTH_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'std'] = float(np.std(QLOG_MONTH_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'low'] = float(np.nanpercentile(QLOG_MONTH_PERSISTENCE['Qlog'],28,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'medium'] = float(np.nanpercentile(QLOG_MONTH_PERSISTENCE['Qlog'],50,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@THREE_MONTH.month & Year==@THREE_MONTH.year'),'high'] = float(np.nanpercentile(QLOG_MONTH_PERSISTENCE['Qlog'],72,axis=0))
        # Store the rescale forecast, mean and std (6-month)
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'6_monthFcst'] = float(ANOMALY_RESCALE_ANALYSIS_PERSISTENCE)
        QLOG_MONTH_06_PERSISTENCE = QLOG_HINDCAST_PERSISTENCE_TS.query("Month == @SIX_MONTH.month")
        
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'Log_Q_PRED_6M'] = float((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][SIX_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][SIX_MONTH.month])
        #RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'Q_PRED_6M'] = float(np.exp((ANOMALY_RESCALE_ANALYSIS_PERSISTENCE*DISCHARGE_MONTHLY_STATS['std'][SIX_MONTH.month])+DISCHARGE_MONTHLY_STATS['mean'][SIX_MONTH.month]))
        
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'mean'] = float(np.mean(QLOG_MONTH_06_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'std'] = float(np.std(QLOG_MONTH_06_PERSISTENCE['Qlog'],axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'low'] = float(np.nanpercentile(QLOG_MONTH_06_PERSISTENCE['Qlog'],28,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'medium'] = float(np.nanpercentile(QLOG_MONTH_06_PERSISTENCE['Qlog'],50,axis=0))
        RESCALE_FORECAST_PERSISTENCE.loc[RESCALE_FORECAST_PERSISTENCE.eval('Month==@SIX_MONTH.month & Year==@SIX_MONTH.year'),'high'] = float(np.nanpercentile(QLOG_MONTH_06_PERSISTENCE['Qlog'],72,axis=0))
        

Dropping year: 1980 - End-Month: 1
Dropping year: 1980 - End-Month: 2
Dropping year: 1980 - End-Month: 3
Dropping year: 1980 - End-Month: 4
Dropping year: 1980 - End-Month: 5
Dropping year: 1980 - End-Month: 6
Dropping year: 1980 - End-Month: 7
Dropping year: 1980 - End-Month: 8
Dropping year: 1980 - End-Month: 9
Dropping year: 1980 - End-Month: 10
Dropping year: 1980 - End-Month: 11
Dropping year: 1980 - End-Month: 12
Dropping year: 1981 - End-Month: 1
Dropping year: 1981 - End-Month: 2
Dropping year: 1981 - End-Month: 3
Dropping year: 1981 - End-Month: 4
Dropping year: 1981 - End-Month: 5
Dropping year: 1981 - End-Month: 6
Dropping year: 1981 - End-Month: 7
Dropping year: 1981 - End-Month: 8
Dropping year: 1981 - End-Month: 9
Dropping year: 1981 - End-Month: 10
Dropping year: 1981 - End-Month: 11
Dropping year: 1981 - End-Month: 12
Dropping year: 1982 - End-Month: 1
Dropping year: 1982 - End-Month: 2
Dropping year: 1982 - End-Month: 3
Dropping year: 1982 - End-Month: 4
Dropping year:

Dropping year: 1999 - End-Month: 7
Dropping year: 1999 - End-Month: 8
Dropping year: 1999 - End-Month: 9
Dropping year: 1999 - End-Month: 10
Dropping year: 1999 - End-Month: 11
Dropping year: 1999 - End-Month: 12
Dropping year: 2000 - End-Month: 1
Dropping year: 2000 - End-Month: 2
Dropping year: 2000 - End-Month: 3
Dropping year: 2000 - End-Month: 4
Dropping year: 2000 - End-Month: 5
Dropping year: 2000 - End-Month: 6
Dropping year: 2000 - End-Month: 7
Dropping year: 2000 - End-Month: 8
Dropping year: 2000 - End-Month: 9
Dropping year: 2000 - End-Month: 10
Dropping year: 2000 - End-Month: 11
Dropping year: 2000 - End-Month: 12
Dropping year: 2001 - End-Month: 1
Dropping year: 2001 - End-Month: 2
Dropping year: 2001 - End-Month: 3
Dropping year: 2001 - End-Month: 4
Dropping year: 2001 - End-Month: 5
Dropping year: 2001 - End-Month: 6
Dropping year: 2001 - End-Month: 7
Dropping year: 2001 - End-Month: 8
Dropping year: 2001 - End-Month: 9
Dropping year: 2001 - End-Month: 10
Dropping year

Dropping year: 2018 - End-Month: 12
Dropping year: 2019 - End-Month: 1
Dropping year: 2019 - End-Month: 2
Dropping year: 2019 - End-Month: 3
Dropping year: 2019 - End-Month: 4
Dropping year: 2019 - End-Month: 5
Dropping year: 2019 - End-Month: 6
Dropping year: 2019 - End-Month: 7
Dropping year: 2019 - End-Month: 8
Dropping year: 2019 - End-Month: 9
Dropping year: 2019 - End-Month: 10
Dropping year: 2019 - End-Month: 11
Dropping year: 2019 - End-Month: 12
Dropping year: 2020 - End-Month: 1
Dropping year: 2020 - End-Month: 2
Dropping year: 2020 - End-Month: 3
Dropping year: 2020 - End-Month: 4
Dropping year: 2020 - End-Month: 5
Dropping year: 2020 - End-Month: 6
Dropping year: 2020 - End-Month: 7
Dropping year: 2020 - End-Month: 8
Dropping year: 2020 - End-Month: 9
Dropping year: 2020 - End-Month: 10
Dropping year: 2020 - End-Month: 11
Dropping year: 2020 - End-Month: 12
Dropping year: 2021 - End-Month: 1
Dropping year: 2021 - End-Month: 2
Dropping year: 2021 - End-Month: 3
Dropping year

In [29]:
RESCALE_FORECAST_PERSISTENCE.head(18)

,Month,Year,Anomaly_Qlog,DATE,1_monthFcst,2_monthFcst,3_monthFcst,6_monthFcst,mean_A_H,std_A_H,mean,std,low,medium,high
0,1,1980,-2.123750,1980-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1980,0.316909,1980-02-01,-2.340416,NaN,NaN,NaN,0.077213,0.940415,2.960569,1.285232,2.103205,2.580642,3.238421
2,3,1980,1.120851,1980-03-01,0.299823,-2.340416,NaN,NaN,0.017155,0.999771,3.109804,1.177222,2.340484,2.653676,3.622124
3,4,1980,1.200881,1980-04-01,1.149825,0.299823,-2.340416,NaN,0.011027,0.965211,3.423551,1.388412,2.479889,3.023710,4.245612
4,5,1980,1.373637,1980-05-01,1.242450,1.149825,0.299823,NaN,0.012194,0.956728,3.781959,1.272669,2.789972,3.752282,4.707521
5,6,1980,0.796236,1980-06-01,1.443303,1.242450,1.149825,NaN,-0.034341,0.975525,4.406921,1.245339,3.696201,4.716933,5.169280
6,7,1980,0.669569,1980-07-01,0.822853,1.443303,1.242450,-2.340416,-0.019906,0.991844,4.629624,0.993045,4.247932,4.812913,5.188282
7,8,1980,0.585540,1980-08-01,0.690822,0.822853,1.443303,0.299823,-0.017168,0.994087,4.513156,1.003208,3.948572,4.521296,5.161298
8,9,1980,-0.481964,1980-09-01,0.604773,0.690822,0.822853,1.149825,-0.016265,0.995094,4.597724,0.966059,3.961631,4.872367,5.124635
9,10,1980,0.640823,1980-10-01,-0.497003,0.604773,0.690822,1.242450,0.013388,0.996679,4.096506,1.261400,3.281784,3.980854,5.105232


In [30]:
RESCALE_FORECAST_PERSISTENCE.tail(18)

,Month,Year,Anomaly_Qlog,DATE,1_monthFcst,2_monthFcst,3_monthFcst,6_monthFcst,mean_A_H,std_A_H,mean,std,low,medium,high
498,7,2021,-0.129605,2021-07-01,1.161775,-0.090161,0.664754,-0.256902,-0.027881,0.983936,4.650095,0.998731,4.247932,4.843983,5.256322
499,8,2021,-0.005090,2021-08-01,-0.132957,1.161775,-0.090161,0.865628,0.003323,0.999779,4.529696,1.008154,3.948572,4.538346,5.161298
500,9,2021,0.059747,2021-09-01,-0.005232,-0.132957,1.161775,0.706709,0.000141,1.000000,4.583139,0.969229,3.961631,4.872367,5.124635
501,10,2021,-1.006693,2021-10-01,0.061410,-0.005232,-0.132957,0.664754,-0.001660,0.999949,4.152996,1.250680,3.386712,3.981831,5.110990
502,11,2021,-1.153155,2021-11-01,-1.048757,0.061410,-0.005232,-0.090161,0.027208,0.985835,3.851357,1.206802,2.983599,3.834529,4.652311
503,12,2021,-1.154836,2021-12-01,-1.206802,-1.048757,0.061410,1.161775,0.031166,0.981371,3.039635,1.121369,2.181597,2.712399,3.584717
504,1,2022,1.141266,2022-01-01,-1.210176,-1.206802,-1.048757,-0.132957,0.032079,0.980778,2.652358,1.152734,1.963947,2.435167,3.289771
505,2,2022,-0.252152,2022-02-01,1.164813,-1.210176,-1.206802,-0.005232,-0.008709,0.987261,2.980340,1.285381,2.103205,2.580642,3.337492
506,3,2022,-0.389803,2022-03-01,-0.284719,1.164813,-1.210176,0.061410,0.032535,0.999887,3.157047,1.193512,2.340484,2.756441,3.714395
507,4,2022,-0.613546,2022-04-01,-0.449192,-0.284719,1.164813,-1.048757,0.049761,0.978568,3.492843,1.406946,2.493991,3.125396,4.286996


Copy variable for Excel

In [31]:
filename_1 = 'santalucia_caudales_HINDCAST_PERSISTENCE_abr2023' + '_FL_1_2_3_6' 

In [32]:
RESCALE_FORECAST_PERSISTENCE.to_csv('../data/' + filename_1 + '.csv', index = False)

In [57]:
#RESCALE_FORECAST_PERSISTENCE.to_clipboard()

In [58]:
#os.makedirs('../results/persistence', exist_ok=True)
#RESCALE_FORECAST_PERSISTENCE.to_csv('../results/persistence/fraymarcos_persistence.csv', index=False)